# Tutorial & Widget Experiments

> This notebook is primarily being used to experiment with how to use the `pymbe` widgets.
>
> It should be fairly "clean" until the `Scratch Pad` section.
>

In the future, this notebook will be cleaned up and renamed "Tutorial".

## 1. Import `pymbe` and create a new user interface

In [1]:
import pymbe.api as pm

In [2]:
pm.UI.new(host_url="http://sysml2-sst.intercax.com")

DockPop(children=(UI(children=(SysML2ClientWidget(children=(Text(value='http://sysml2-sst.intercax.com', descr…

## 2. Use the widget
...or automatically load the `Kerbal` model using the cell below by:

1. Grabbing the individual widgets

In [3]:
ui, *_ = _.children
client, tree, inspector, lpg = ui.children

2. load a small model

In [4]:
client.project_selector.value = client.project_selector.options["12b-Allocation"]
client._download_elements()

...and, for example, select some edge types to filter down the diagram

In [ ]:
EDGE_TYPES_TO_SELECT =  ("Superclassing", "FeatureTyping", "FeatureMembership")
lpg.edge_type_selector.value = [
    edges
    for key, edges in lpg.edge_type_selector.options.items()
    if any(
        key.startswith(edge_type)
        for edge_type in EDGE_TYPES_TO_SELECT
    )
]
lpg._update_diagram_graph()

# Interpretation

> This is where we are refactoring the $M_0$ interpretation...

In [ ]:
def safe_get(data: dict, *keys, default=None):
    for key in keys:
        if key not in data:
            return default
        data = data[key]
    return data

safe_get(lpg.elements_by_id, "cb4386fe-67d9-4743-995f-5d4a4162e26e", "owner", "@id")

In [ ]:
UNSPECIFIED = object()

class SafeDict(dict):

    def __init__(self, *args, default=None, **kwargs):
        self.__default__ = default or {}
        super().__init__(*args, **kwargs)
        for key, value in self.items():
            self[key] = SafeDict.__convert(value)

    def __setitem__(self, k, v):
        dict.__setitem__(self, k, SafeDict.__convert(v))

    def get(self, item, default=None):
        return super().get(item, default) or self.__default__

    @staticmethod
    def __convert(o):
        """
        Recursively convert `dict` objects in `dict`, `list`, `set`, and
        `tuple` objects to `attrdict` objects.
        """
        if isinstance(o, dict):
            o = SafeDict(o)
        elif isinstance(o, list):
            o = list(SafeDict.__convert(v) for v in o)
        elif isinstance(o, set):
            o = set(SafeDict.__convert(v) for v in o)
        elif isinstance(o, tuple):
            o = tuple(SafeDict.__convert(v) for v in o)
        return o


a = SafeDict(dict(agh=1, bum=2, bar=3, baz=dict(bar=2,baz=dict(bar=3,baz=4))))
a["as"] = "if"
a.get("as"), type(a.get("baz")), a.get("baz")

In [ ]:
from pymbe.interpretation import *

def roll_up_upper_multiplicities(lpg):
    banded_featuring_graph = make_banded_featuring_graph(lpg)

    banded_roots = [
        banded_featuring_graph.nodes[node]
        for node in banded_featuring_graph
        if banded_featuring_graph.out_degree(node) == 0
    ]
    
    for part_usage in 

elements = lpg.elements_by_id

features = [
    id_
    for id_, data in elements.items()
    if data["@type"] == "Feature"
]

for feature in features:
    upper_multiplicity = get_feature_upper_multiplicity(elements, feature)
    if upper_multiplicity:
        print(elements[feature]["qualifiedName"], upper_multiplicity)

# Troubleshooting

## Fix linking issue with diagram element selector

In [4]:
self = lpg.diagram
import networkx as nx

_, hierarchy = self.elk_app.transformer.source

try:
    item = lpg.diagram.elk_app.selected[0]
    res = next(hierarchy.predecessors(item)).node
except nx.NetworkXError:
    res = item.data.get("properties", {}).get("@id")

res.id, res.data["@id"]

IndexError: tuple index out of range

In [ ]:
tree.selected, explorer.selected, lpg.selected

In [5]:
lpg.diagram.elk_app.diagram.selected

('07aee1b6-3400-4d86-8a9f-ce869d0c9628',
 '8b6e0ff4-292c-4c6f-b3b8-214528916f20')

In [6]:
lpg.selected

()

In [ ]:
self = lpg.diagram

_, hierarchy = self.elk_transformer.source

self.parts[self.selected[0]], self.elk_app.selected[0]

for selected_id in self.selected:
    node_selected = self.parts.get(selected_id)
    print(node_selected.id)

In [ ]:
node_selected.children[0] == self.elk_app.selected[0].node

---------------

In [42]:
from dataclasses import dataclass, field


@dataclass
class Mapper:
    to_map: dict = field(repr=False)
    from_map: dict = field(default=None, repr=False)
        
    def __repr__(self):
        return f"Mapper({len(self.to_map)} Items)"

    def __post_init__(self, *args, **kwargs):
        self.from_map = {v: k for k, v in self.to_map.items()}

    def get(self, *items):
        found = [
            self.to_map.get(item, self.from_map.get(item))
            for item in items
        ]
        if len(found) < 2:
            found = found[0]
        return found

mapper = Mapper(from_elk_id)

mapper.get(*self.elk_app.diagram.selected)

['8e7fd118-6c08-4766-87c1-21f036a337b3',
 '722b09a9-bd47-456a-a2fd-bdb01d3fb004']

In [43]:
a = {1,2,}
a.symmetric_difference

{1, 2}

In [5]:
self = lpg.diagram

In [6]:
relationships, hierarchy = self.elk_app.transformer.source

In [27]:
transformer = self.elk_app.transformer
relationships, hierarchy = transformer.source

import ipyelk
from ipyelk.contrib.elements import (
    Compartment,
)

def id_from_item(item):
    id_ = None
    if isinstance(item, ipyelk.transform.Edge):
        id_ = item.data.get("properties", {}).get("@id")
    elif isinstance(getattr(item, "node", None), Compartment):
        id_ = next(hierarchy.predecessors(item)).node.id
    if id_ is None:
        self.log.debug(f"Could not parse: {item}")
    return id_

from_elk_id = {
    elk_id: id_from_item(elk_item)
    for elk_id, elk_item in transformer._elk_to_item.items()
}

from_elk_id = {
    elk_id: sysml_id
    for elk_id, sysml_id in from_elk_id.items()
    if sysml_id is not None
}
found_ids = set(from_elk_id.values())

for extra in set(lpg.elements_by_id).symmetric_difference(found_ids):
    print(lpg.elements_by_id[extra])




{'@context': {'@base': 'http://sysml2-sst.intercax.com:9000/projects/021a95a5-244d-45fc-91c6-c1ff9faa6a54/commits/50b4656c-1348-4782-92e1-8bc35b2d5912/elements/', '@import': 'http://sysml2-sst.intercax.com:9000/jsonld/EndFeatureMembership.jsonld'}, '@type': 'EndFeatureMembership', '@id': '46a83e4e-c6ea-47de-9855-00634481bf86', 'aliasId': [], 'direction': None, 'documentation': [], 'documentationComment': [], 'featureOfType': {'@id': '8e7fd118-6c08-4766-87c1-21f036a337b3'}, 'featuringType': None, 'humanId': None, 'identifier': '46a83e4e-c6ea-47de-9855-00634481bf86', 'isComposite': False, 'isDerived': False, 'isPort': False, 'isPortion': False, 'isReadOnly': False, 'memberElement': {'@id': '8e7fd118-6c08-4766-87c1-21f036a337b3'}, 'memberFeature': {'@id': '8e7fd118-6c08-4766-87c1-21f036a337b3'}, 'memberName': None, 'membershipOwningNamespace': {'@id': '8c9299d8-b198-417e-96ac-6b28a2e4cd1a'}, 'name': None, 'ownedAnnotation': [], 'ownedElement': [], 'ownedMemberElement': {'@id': '8e7fd118-6

In [21]:
transformer.from_id("61bd9433-3eec-47f7-af7e-3bf71f19071b").node

PartContainer(labels=[], properties={}, layoutOptions={}, shape=None, ports={}, children=[Part(labels=[], properties={}, layoutOptions={'org.eclipse.elk.hierarchyHandling': 'INCLUDE_CHILDREN', 'org.eclipse.elk.padding': '[top=0.0,bottom=0.0,left=0.0,right=0.0]', 'org.eclipse.elk.spacing.nodeNode': '0.0', 'org.eclipse.elk.edge.edgeNode': '0.0', 'org.eclipse.elk.aspectRatio': '100.0', 'org.eclipse.elk.expandNodes': 'true', 'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_CENTER INSIDE', 'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE', 'org.eclipse.elk.spacing.componentComponent': '0.0'}, shape=None, ports={}, children=[Compartment(labels=[Label(labels=[], properties={'cssClasses': 'compartment_title_1'}, layoutOptions={'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_CENTER INSIDE'}, shape=None, text='«ActionUsage»'), Label(labels=[], properties={'cssClasses': 'compartment_title_2'}, layoutOptions={'org.eclipse.elk.nodeLabels.placement': 'H_CENT

In [9]:
missing = [elk_id for elk_id, our_id in from_elk_id.items() if our_id is None]
lpg.diagram.elk_app.diagram.selected = missing

In [10]:
from_elk_id

{'8a260fce-aebb-44f8-a99c-01aceb184316': '1709a68e-ea57-4e8b-b073-8c416f934233',
 'a0033005-0dbb-458a-b950-ef32938fe589': '1709a68e-ea57-4e8b-b073-8c416f934233',
 'a40b584c-5919-46be-b62a-86e0e355fc7c': '4020dc4e-b2ae-4aea-add1-c2834428630c',
 '76530180-b14c-45bb-b811-0bb355702673': '4020dc4e-b2ae-4aea-add1-c2834428630c',
 'e0d59e9c-f9c0-4778-a6d8-669b75cb16d2': '5aaad7a1-a8ef-4d64-be36-8a9ac3808644',
 'dc5a1352-96d4-4792-b62f-89d6c7f77a46': '5aaad7a1-a8ef-4d64-be36-8a9ac3808644',
 '393b30a7-c4be-42cc-bda0-c38c8e89a3bc': 'b677ae1e-d891-4272-8c7c-ec6a7c07b05e',
 '7496f93e-a72c-4960-a8f3-2e6654709ece': 'b677ae1e-d891-4272-8c7c-ec6a7c07b05e',
 '98102ee9-12c4-446b-bfbe-8a61764a3eff': 'bfc96de3-40da-4965-ab4c-6fe92377d63b',
 '82e28110-d014-47e6-a304-2e6a99837cfd': 'bfc96de3-40da-4965-ab4c-6fe92377d63b',
 'eb692b61-8f74-4daa-b531-dbc07d0d1b59': '8e7fd118-6c08-4766-87c1-21f036a337b3',
 '57671b71-de5d-4e1b-9583-0df3dce26d33': '8e7fd118-6c08-4766-87c1-21f036a337b3',
 'b3493db0-1cbc-4bd6-bae8-14

# Scratch Pad
> **WARNING**: Anything below this point is just for experimentation purposes

## RDF Experiments

In [ ]:
def process_edge(source, target, edge_type, data=None):
    if edge_type in ("FeatureTyping", "FeatureMembership"):
        source, target = target, source
    return [source, target, data]

graph = nx.DiGraph()
graph.add_edges_from([
    process_edge(source, target, edge_type, data)
    for (source, target, edge_type), data in dict(client.lpg.graph.edges).items()
    if edge_type in ("Superclassing", "FeatureTyping", "FeatureMembership")
])
graph

In [ ]:
diagram.graph = graph

In [ ]:
diagram = client.lpg.make_diagram(
    graph=client.lpg.subgraph(edge_types=("Superclassing", "FeatureTyping^-1", "FeatureMembership^-1"))
)
elk_app, *_ = diagram.children
diagram

In [ ]:
# Select the root node in the diagram...
# ... or you can manually select one yourself
elk_app.selected = "5260380b-6fda-43cc-993f-5df58868edbb",

In [ ]:
first_element_selected, *_ = elk_app.selected
client.elements_by_id[first_element_selected]

# Parse JSON-LD into RDF

In [ ]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

result = client.rdf.graph
# result = g.parse(url, format='turtle')

G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
pos = nx.spring_layout(G, scale=2)
edge_labels = nx.get_edge_attributes(G, 'r')

In [ ]:
ax = plt.figure(figsize=(50,30)).gca();
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
nx.draw(G, with_labels=True, ax=ax)

# TODOs
1. Finish fleshing out the process in the `Kerbal Model.ipynb`
2. Modify the subgraph generator so it can take the value from the `Type Selector` directly
3. Improve the ipyelk diagram widget (may need to make improvements to `ipyelk`)
   * Add arrows
   * Add compartments
   * Fix layout
   * Add widget to see node details
4. Finalize the RDF formulation